# Section 1: Env Setup
- Load data in tar file
- Mount Google Drive
- pip upgrade matplotlib dependencies
- Imports

## ENV

In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


In [ ]:
!cp /content/gdrive/MyDrive/colab_output/project_0123.tar.gz /content/
!tar -xf /content/project_0123.tar.gz

In [ ]:
! pip install --upgrade matplotlib # ONLY run this cell upon intially opening the notebook

In [ ]:
# ! wget https://www.mydrive.ch/shares/39258/39d0eaefc4dc611048ec970f886bad8d/download/420939224-1629955498/mvtec_screws_v1.0.tar.gz

In [ ]:
#@title
# !tar -xf /content/data/mvtec_screws_v1.0.tar.gz
# !unzip /content/data/masks.zip

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
%aimport src.utils.box_cutter
%aimport src.utils.funcs
%aimport src.utils.classes
%aimport src.models.models
%aimport src.models.layers


## Imports

In [ ]:
import os
import json
from pathlib import Path
import pprint as pp
from glob import glob, iglob
from PIL import Image, ImageFilter, ImageDraw
from typing import List, Union, Tuple, BinaryIO
import shutil

import pickle
import tensorflow as tf
from tensorflow.keras import layers, losses, metrics
from tensorflow.keras.models import Model
from tensorflow.keras.utils import image_dataset_from_directory, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.patches import Rectangle, Polygon, Circle

import numpy as np
import pandas as p230
from tabulate import tabulate
from IPython.display import HTML


```
data_img_path = Path('/content/data/images')
data_ann_path = Path('/content/data/mvtec_screws.json')
coco = COCO(data_ann_path)
img_ids = coco.getImgIds()
imgs = coco.loadImgs(img_ids[-3:])
```

In [ ]:
#@title
pp.PrettyPrinter(indent=4)

## Configure Color Picker

In [ ]:
COLORS = list(mcolors.TABLEAU_COLORS.keys())
def a_color(a_range:int, colors:list=COLORS):
    for _ in range(a_range):
        yield np.random.choice(colors)


# Create Train and Test Labels


## Gather Train/Val/Test Info

In [ ]:
train_file = Path("/content/mvtec_screws_train.json")
X_train_path = Path('/content/bbox/train/images')
y_train_path = Path("/content/bbox/train")
train_db = COCO(train_file)

train_ids = train_db.getImgIds()
train_imgs = train_db.loadImgs(train_ids)
print(f"Found {len(train_imgs)} training images")

In [ ]:
val_file = Path("/content/mvtec_screws_val.json")
X_val_path = Path('/content/bbox/val/images')
y_val_path = Path("/content/bbox/val")
val_db = COCO(val_file)

val_ids = val_db.getImgIds()
val_imgs = val_db.loadImgs(val_ids)
print(f"Found {len(val_imgs)} validation images!")

In [ ]:
test_file = Path("/content/mvtec_screws_test.json")
X_test_path = Path('/content/bbox/test/images')
y_test_path = Path("/content/bbox/test")
test_db = COCO(test_file)

test_ids = test_db.getImgIds()
test_imgs = test_db.loadImgs(test_ids)
print(f"Found {len(test_imgs)} test images")

In [ ]:
ann_id = val_db.getAnnIds(img['id'])
annot = val_db.loadAnns(ann_id)

In [ ]:
class data_generator:
    def __init__(self,
                 X_path:str,
                 y_path:str, 
                 categories:List[int],
                 input_shape:Tuple[int, int],
                 target_size:Union[Tuple[int,], Tuple[int, int], None]=None,
                 batch_size=32):
        """

        """         
        self.X_path = self.clean_path(X_path)
        self.y_path = self.clean_path(y_path)
        self.input_shape = input_shape
        self.target_size = self._compute_target_size(target_size)
        self.X_stack = self._get_X()
        self.y_stack = self._get_y()
        
    
    def clean_path(self, path):
        if path.endswith('/'):
            return path[:-1]
    
    def _compute_target_size(self, target_size):
        if target_size == (0,):
            target = target_size[0]
            a_val = self.input_shape[0]
            b_val = self.input_shape[1]

            a_target = np.sqrt(np.multiply(target**2, a_val) / b_val)
            b_target = np.sqrt(np.multiply(target**2, b_val) / a_val)

            output = (round(a_target), round(b_target))
            return output
        try:
            assert(len(target_size) == 2 or target_size is None)
        except AssertionError as err:
            print("target_size must be in the form Tuple[int,], Tuple[int, int] or None")
            raise
        return target_size
            
    def _get_X(self) -> np.ndarray:
        # get my images
        #   use the list of images to get 
        #   labels and form them into an
        #   array of class labels
        X_set = np.empty((0,), dtype=np.float32)
        print(f"Working on grabbing images...")
        for file in iglob(f"{self.X_path}/*"):
            X_img = np.asarray(load_img(file, target_size=self.target_size), dtype=np.float32)
            if X_set.shape == (0,):
                X_set = np.empty(shape=(0,) + X_img.shape, dtype=np.float32)
            X_set = np.append(X_set, X_img.reshape((1,) + X_img.shape), axis=0)
            if len(X_set) % 10 == 0:
                print("[]", end="")
        print(f"\n\nReturning Image stack with shape: {X_set.shape}")
        return X_set

    def _get_y(self) -> np.ndarray:
        y_set = np.empty((0,), dtype=np.int8)
        print("\n\nCompiling mask layers for each image...")
        for x_file in iglob(f"{self.X_path}/*"):
            y_mask = np.empty((0,), dtype=np.int8)

            for y_file in iglob(f"{self.y_path}/**/*{os.path.basename(x_file)}"):
                y_layer = np.asarray(load_img(y_file, color_mode="grayscale", target_size=self.target_size), dtype=np.float32)
                y_layer = y_layer.reshape((1,) + y_layer.shape + (1,))

                if y_mask.shape == (0,):
                    y_mask = y_layer
                    continue
                y_mask = np.append(y_mask, y_layer, axis=3)

            if y_set.shape == (0,):
                y_set = np.empty((0,) + y_mask.shape[1:], dtype=np.int8)
            y_set = np.append(y_set, y_mask, axis=0)
            if len(y_set) % 10 == 0:
                print("[]", end="")
        print(f"\n\nReturning mask stack with shape: {y_set.shape}")
        return y_set


In [ ]:
def rotate(input, center, phi):
    vector = np.subtract(input, center)
    id = np.asarray([[np.cos(phi), -np.sin(phi)],
                     [np.sin(phi), np.cos(phi)]], 
                    dtype=np.float32)
    vector = np.matmul(id, vector)
    return np.add(vector, center)

def get_bbox(file_name):
    for img in val_imgs:
        if img['file_name'] == file_name:
            ann_id = val_db.getAnnIds(img['id'])
            annot = val_db.loadAnns(ann_id)
            bbox_truth = make_masks(annot, (1440, 1920), draw_bbox=False, draw_center=True)
            print(bbox_truth.shape)
    return bbox_truth 

def process_bbox(data:list) -> np.ndarray:
    results = np.zeros((1, 14, 2, 2), dtype=np.float32)
    center_line = np.empty((0,), dtype=np.float32)
    for cat in range(1, 14, 1):
        entry_bboxes = np.empty((0, 2, 2), dtype=np.float32)
        for entry in data: 
            bbox = np.zeros((1, 2, 2), dtype=np.float32)
            if entry['category_id'] == cat:
                # grab bbox info
                row, col, width, height, phi = entry['bbox']
                # define center point of bbox
                center = np.array([col, row])
                # -pi to pi -> 0 to 2*pi 
                phi = -1 * (phi - np.pi)
                # initial bounds
                y0, y1 = row-height/2, row+height/2
                x0, x1 = col-width/2, col+width/2
                # corner points
                # corners = [(x0, y0), (x0, y1), (x1, y1), (x1, y0)] # Corners
                corners = [(x0, y0), (x1, y1)]
                # rotate_box([p0, p1, p2, p3], center, phi)
                bbox = [rotate(point, center, phi) for point in corners]
                input_size = (1440, 1920)
                target_size = (192, 256)
                for i, (x, y) in enumerate(bbox):
                    x = x * target_size[1] / input_size[1]
                    y = y * target_size[0] / input_size[0]
                    bbox[i] = (x, y)
                    
                # rotated corners
                bbox = np.array(bbox, dtype=np.float32)
                bbox = bbox.reshape((1,) + bbox.shape)
                print(bbox.shape)
                # bpoints = np.array([np.mean([rp0, rp1], axis=0), np.mean([rp2, rp3], axis=0)])
            entry_bboxes = np.append(entry_bboxes, bbox, axis=0)
        print(entry_bboxes.shape)
        print(results.shape)
         
        results = np.append(results, entry_bboxes, axis=0)
    
    return bbox_points

    
val_image = "screws_099.png"
img_array = np.asarray(load_img(f"/content/images/val/{val_image}", target_size=(192, 256)), dtype=np.float32) / 255
plt.imshow(img_array)
bboxes = get_bbox(val_image)
print(bboxes.shape)
for xy in bboxes:
    x, y = [[x for x, y in xy], [y for x, y in xy]]
    x.append(x[0])
    y.append(y[0])
    plt.plot(x, y)
plt.axis('off')
plt.show()

### Problems
* Photos have multiple objects belonging to the same class
* Photos have uneven amounts of objects and classes

### Solutions
1. Build a SSD or some form of Fast R-NN network.
    - Pros
        * This is the correct way to do it.
    - Cons
        * I don't know how to do this and it's not simple.
        * It's almost certianly impossible in the time I have left.
2. Collapse label masks into one layer with class values as floats and then use the CNN structure you already have to predict it. 
    - Pros
        * I could probably have this ready to start training today.
    - Cons:
        * This would probably not work super well. 
3. Don't predict bounding boxes or segementation just use a vgg16 classifier into a fully connected block to predict class labels for each image.
    - Pros
        * It's pretty simple, and easy.
    - Cons
        * It's very boring and would require a lot of reworking of the data. Probably not a huge amount but a nontrivial amount

4. Build an unsupervised Segmentation model
    - Pros
        * This is challenging and interesting
    - Cons
        * I probably can't do this
        * It's a complicated and difficult problem. There are no ready made solutions.


In [ ]:
batch_gen = data_generator("/content/images/train/", y_path="/content/bbox/train/", categories=[], input_shape=(1440, 1920), target_size=(192, 256))    

In [ ]:
# # flip = batch_gen.y_stack.copy() * -1
# flip[0,150,160,:]
# batch_gen.y_stack = flip
print(tabulate(batch_gen.y_stack[0,135:150, 140:155,0]))

In [ ]:
plt.imshow(batch_gen.X_stack[0] / 255)
plt.imshow(batch_gen.y_stack[0,:,:,0] / 255, cmap='gray', alpha=.4)
plt.vlines(x=147, ymax=160, ymin=130, colors='red')
plt.hlines(y=142, xmin=130, xmax=175, colors='red')

```python
writer = PillowWriter(fps=2)
fig, ax = plt.subplots(figsize=(8,5))
def animate(i):
    ax.imshow(batch_gen.X_stack[0]/255)
    ax.imshow(batch_gen.y_stack[0,:,:,i] * -1, alpha=.5, cmap='gray')

anim = FuncAnimation(fig, animate, frames=list(range(12)))
anim.save("/content/mask_anim02_2fps.gif", writer, dpi=300)
```

## Define Draw Bounding Box

In [ ]:
def draw_box(data:dict, axis:plt.Axes):
  for entry in data: 
    row, col, width, height, phi = entry['bbox']
    angle = (180/np.pi) * -phi
    x = col - width/2
    y = row - height/2
    bbox = axis.add_patch(
        Rectangle((x,y), 
                  width,
                  height,
                  angle,
                  rotation_point='center',
                  edgecolor='green',
                  facecolor=a_color(),
                  linewidth=4,
                  alpha=.3))
  return axis

## Image with Bbox Annotation

```python
fig, ax = plt.subplots(len(imgs), 2, figsize=(10, 12))
for img, ax_n, in zip(imgs, ax):
  ann_id = coco.getAnnIds([img['id']])
  ann_data = coco.loadAnns(ann_id)
  i_n = Image.open(f"/content/data/images/{img['file_name']}")
  ax_n[0].imshow(i_n)
  ax_n[0].axis('off')
  ax_n[1].imshow(i_n)
  ax_n[1] = draw_box(ann_data, ax_n[1])
  ax_n[1].axis('off')
fig.tight_layout()
plt.show();
```

## Copy Files into Separate Directories

```python
for img in test_imgs:
  shutil.copyfile(f"/content/data/images/{img['file_name']}",
                  f"/content/data/test/images/{img['file_name']}")
  shutil.copyfile(f"/content/data/bbox/{img['file_name']}", 
                  f"/content/data/test/bbox/{img['file_name']}")
  shutil.copyfile(f"/content/data/bbox_centers/{img['file_name']}", 
                  f"/content/data/test/center/{img['file_name']}")
```

```python
for img in val_imgs:
  shutil.copyfile(f"/content/data/full_set/images/{img['file_name']}",
                  f"/content/data/val/images/{img['file_name']}")
  shutil.copyfile(f"/content/data/full_set/bbox_mask/{img['file_name']}", 
                  f"/content/data/val/bbox/{img['file_name']}")
  shutil.copyfile(f"/content/data/full_set/bbox_centers/{img['file_name']}", 
                  f"/content/data/val/center/{img['file_name']}")
```

```python
for img in train_imgs:
  shutil.copyfile(f"/content/data/images/{img['file_name']}",
                  f"/content/data/train/images/{img['file_name']}")
  shutil.copyfile(f"/content/data/bbox_mask/{img['file_name']}", 
                  f"/content/data/train/bbox/{img['file_name']}")
  shutil.copyfile(f"/content/data/bbox_centers/{img['file_name']}", 
                  f"/content/data/train/center/{img['file_name']}")
```

## Helper Functions For Making Masks

In [ ]:
def circle(radius, center_point):
  circle = []
  c_x, c_y = center_point
  while radius > 0:
    circ = round(radius**2 * np.pi)
    for pixel in range(circ):
      x, y = c_x + radius*np.cos(pixel * (2*np.pi)/circ),c_y + radius*np.sin(pixel * (2*np.pi)/circ) 
      circle.append((x, y))
    radius -= 1
  return circle
  
def rotate_point(vector, center, phi):
  x, y = vector
  i, j = center
  vector = np.array([x-i, y-j], dtype=np.float32)
  id = np.asarray([[np.cos(phi), -np.sin(phi)],
                  [np.sin(phi), np.cos(phi)]], 
                  dtype=np.float32)
  x, y = np.matmul(id, vector)
  return np.array([x+i, y+j], dtype=np.float32)

def rotate_box(box_edges:np.ndarray, center, phi):
  for i, vec in enumerate(box_edges):
    vec = rot_point(vec, center, phi)
    box_edges[i] = vec
    
  return box_edges

## Define Mask Making Function

In [ ]:
def make_masks(data:dict,
               input_size:tuple=(None, None),
               target_size:tuple=(None, None),
               draw_bbox:bool=True,
               draw_center:bool=False,
               save_mask:bool=False, 
               file_name:str=None,
               path_to_dir:str=None) -> np.ndarray:
  """
  Function takes an image shape and a dictionary that defines a bounding box and
  returns a numpy array version of the image mask.
  """
  colors = ['red', 'blue', 'green', 'purple']
  pick = np.random.choice
  count = 0
  result = np.empty(size + (0,), dtype=np.float32)
  for cat in range(1, 14, 1):
    mask = Image.new("L", tuple(reversed(size)), color='black')
    for entry in data: 
      if entry['category_id'] == cat:
        draw = ImageDraw.Draw(mask, "L")
        # grab bbox info
        row, col, width, height, phi = entry['bbox']
        # define center point of bbox
        center = np.array([col, row])
        # -pi to pi -> 0 to 2*pi 
        phi = phi + np.pi
        # initial bounds
        y0, y1 = row-height/2, row+height/2
        x0, x1 = col-width/2, col+width/2
        
        # corner points
        # corners = [(x0, y0), (x0, y1), (x1, y1), (x1, y0)] # Corners
        corners = [(x0, y0), (x1, y1)]
        # rotate_box([p0, p1, p2, p3], center, phi)
        bbox = [rotate(point, center, phi) for point in corners]
        input_size = (1440, 1920)
        target_size = (192, 256)
        for i, (x, y) in enumerate(bbox):
            x = x * target_size[1] / input_size[1]
            y = y * target_size[0] / input_size[0]
            bbox[i] = (x, y)
        
        # draw mask shapes
        if draw_bbox:
          draw.polygon([rp0, rp1, rp2, rp3], fill='white')
        if draw_center:
          draw.polygon(circle(15, center), fill='white')
    
    if save_mask:    
        mask.save(f"{path_to_dir}/{cat:02d}_{file_name}",
                  format='png')
    mask_data = np.asarray(mask, dtype=np.float32)
    result = np.append(result, mask_data.reshape(size + (1,)), axis=2)
  
  return result

## Make the Masks

```python
# fig, axs = plt.subplots(len(img_set[5:]), 2, figsize=(10, 20))
img_dbs = {'val':val_db, 'test':test_db} # Done: 'train':train_db, 
img_ids = {'val':val_ids, 'test':test_ids} # Done: 'train':train_ids, 
for div, img_db in img_dbs.items():
  img_set = img_db.loadImgs(img_ids[div])
  for img in img_set:

    arr_img = np.asarray(
                    Image.open(f"/content/data/{div}/images/{img['file_name']}"),
                    dtype=np.float32)

    box_data = img_db.loadAnns(img_db.getAnnIds(img['id'])) 


    make_masks(box_data, size=arr_img.shape[:-1], # mask_bboxes = 
                        draw_bbox=True,
                        draw_center=False,
                        save_mask=True,
                        file_name=img['file_name'],
                        path_to_dir=f"/content/data/{div}/bbox_by_cat/")
    
    make_masks(box_data, size=arr_img.shape[:-1], # mask_centers = 
                        draw_bbox=False,
                        draw_center=True,
                        save_mask=True,
                        file_name=img['file_name'],
                        path_to_dir=f"/content/data/{div}/cent_by_cat/")
  #   break
  # break

# fig, ax = plt.subplots(figsize=(8, 5))

# def animate(i):
#   ax.clear
#   ax.imshow(arr_img / 255)
#   ax.imshow(mask_bboxes[:,:,i], cmap='gray', alpha=.5)

# writer = PillowWriter(fps=2)
# ax.axis('off')
# frames = list(range(13))
# anim = FuncAnimation(fig, animate, frames=frames, save_count=13)
# anim.save("/content/mask_anim_2fps.gif", writer, dpi=72)
# plt.close()
# print(f"mask layer shapes: {mask_bboxes.shape}")
```

# Baseline Model Definition

## Layer Definition

### Convolution Block Definition

In [ ]:
def conv2d_block(x, filters, kernel_size=3, reps:int=2, pooling:bool=False, **kwargs):
  residual = x
  options = {}
  if kwargs:
    options.update(**kwargs)
  for rep in range(reps):
    if not rep:
      options.update({'strides': 2})
    else:
      options['strides'] = 1
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(filters, kernel_size, padding="same", use_bias=False, **options)(x)
  
  if pooling:
    x = layers.MaxPooling2D(kernel_size, strides=2, padding="same")(x)
    # residual = layers.Conv2D(filters, 1, strides=2)(residual)
  # elif filters != residual.shape[-1]:
  #   residual = layers.Conv2D(filters, 1)(residual)
  
  # x = layers.add([x, residual])
  return x

### Conv-Transpose (deconvolution) Layer Block Definition

In [ ]:
def conv2d_T_block(x, filters, kernel_size=3, reps:int=2, **kwargs):
  residual = x
  options = {}
  if kwargs:
    options.update(**kwargs)
  for rep in range(reps):
    if not rep:
      options.update({'strides': 2})
    else:
      options['strides'] = 1
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2DTranspose(filters, kernel_size, padding="same", use_bias=False, strides=2)(x)
  
  # residual = layers.Conv2D(filters, 1)(residual)
  
  # x = layers.add([x, residual])
  return x

## Model Definition

In [ ]:
def get_model(input_shape, num_classes, filter_blocks:List, rescaling:bool=False, **kwargs):
  inputs = tf.keras.Input(shape=input_shape)

  x = layers.Rescaling(1./255)(inputs)
    
  for block in filter_blocks:
    x = conv2d_block(x, block, pooling=True, **kwargs)
  
  r_filter_blocks = reversed(filter_blocks)
  for t_block in r_filter_blocks:
    x = conv2d_T_block(x, t_block, **kwargs)

  # x = layers.GlobalAveragePooling2D()(x)
  outputs = layers.Conv2D(num_classes, 3, activation='softmax', padding='same', name="output_layer")(x)

  model = tf.keras.Model(inputs, outputs)
  return model

In [ ]:
input_shape = batch_gen.X_stack[0].shape
print(input_shape)


In [ ]:
model = get_model(input_shape, 13, [39, 78, 156], rescaling=True)
model.summary()

In [ ]:
batch_gen.X_stack[0].shape

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')
callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=.05, patience=10)

In [ ]:
history = model.fit(batch_gen.X_stack, batch_gen.y_stack / 255,
                    batch_size=5,
                    callbacks = [callbacks],
                    epochs=150)

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
pred = model.predict(batch_gen.X_stack[60:67])
fig, axs = plt.subplots(13, 6, figsize=(12, 26))
for j in range(13):
    for i, ax in zip(range(6), axs[j]):
        ax.imshow(pred[i,:,:,j])
        ax.axis('off')
fig.tight_layout()
plt.savefig('/content/base_model_output.png')
plt.show()

In [ ]:
compare = [batch_gen.X_stack[60:67] / 255, pred[:,:,:,2], np.max(batch_gen.y_stack[60:67] / 255, axis=3)]
fig, axs = plt.subplots(3, 7, figsize=(21, 9))
for j, itm in enumerate(compare):
    for i, ax in enumerate(axs[j]):
        ax.imshow(itm[i])
        ax.axis('off')
fig.tight_layout()
plt.savefig('/content/base_model_compare.png')
plt.show()

In [ ]:
pred.shape

# Data Organization

# Configuring a Datagenerator

In [ ]:
test_data = data_generator('/content/images/test/', '/content/bbox/test/', list(range(1, 14, 1)), (1440, 1920), (192, 256))
val_data =  data_generator('/content/images/val/', '/content/bbox/val/', list(range(1, 14, 1)), (1440, 1920), (192, 256))


In [ ]:
train_data = data_generator('/content/images/train/', '/content/bbox/train/', list(range(1, 14, 1)), (1440, 1920), (192, 256)) 

In [ ]:
def get_class_symbol(data:List[dict]) -> np.ndarray:
    bbox_points = np.empty((0,), dtype=np.float32)
    center_line = np.empty((0,), dtype=np.float32)
    for cat in range(1, 14, 1):
        for entry in data: 
            if entry['category_id'] == cat:
            
                # grab bbox info
                row, col, width, height, phi = entry['bbox']
                
                # define center point of bbox
                center = np.array([col, row])
                # -pi to pi -> 0 to 2*pi 
                phi = phi + np.pi
        
                # initial bounds
                y0, y1 = row-height/2, row+height/2
                x0, x1 = col-width/2, col+width/2
                
                # corner points
                p0, p1, p2, p3 = (x0, y0), (x0, y1), (x1, y1), (x1, y0)
        
                # rotation
                # rotate_box([p0, p1, p2, p3], center, phi)
                rx0, ry0 = rotate(p0, center, -phi)
                rx1, ry1 = rotate(p1, center, -phi)
                rx2, ry2 = rotate(p2, center, -phi)
                rx3, ry3 = rotate(p3, center, -phi)
        
                # rotated corners
                rp0 = (rx0, ry0)
                rp1 = (rx1, ry1)
                rp2 = (rx2, ry2)
                rp3 = (rx3, ry3)
                
                # draw mask shapes
                bbox = np.array([rp0, rp1, rp2, rp3])
                bpoints = np.array([np.mean([rp0, rp1], axis=0), np.mean([rp2, rp3], axis=0)])

                yield bbox, bpoints



In [ ]:
fig, axs = plt.subplots(6, figsize=(8, 20))
colors = a_color(100)
for i, ax in zip(range(6), axs):
    cur_img = np.asarray(Image.open(f"/content/images/val/{val_imgs[i]['file_name']}"), dtype=np.float32)
    ax.imshow(cur_img / 255)
    ann_id = val_db.getAnnIds(val_imgs[i]['id'])
    annot = val_db.loadAnns(ann_id)
    points = get_class_symbol(annot)
    for b, p in points:
        color = next(colors)
        # ax.add_patch(Polygon(b, color=color))
        # print(b[0], b[1], b[2], b[3])
        x, y = [[x for x,y in b], [y for x, y in b]]
        x.append(x[0])
        y.append(y[0])
        ax.plot(x, y, color=color)
        x0, y0 = p[0]
        x1, y1 = p[1]
        ax.plot([x0, x1], [y0, y1], color=color)

    ax.axis('off')
fig.tight_layout()
plt.show()


    